In [1]:
# 라이브러리 임포트 및 eager 모드 실행
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

In [3]:
# 데이터 셋
X_train = [[1., 2.], [2., 3.], [3., 1.], [4., 3.], [5., 3.], [6., 2.]]
y_train = [[0.], [0.], [0.], [1.], [1.], [1.]]

X_test = [[5., 2.]]
y_test = [[1.]]

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(len(X_train))

In [20]:
# 가중치 & 편향
W = tf.Variable(tf.zeros([2, 1]), name = 'weight')
b = tf.Variable(tf.zeros([1]), name = 'bias')

In [28]:
# 로지스틱 함수
def logistic_regression(features):
    hypothesis = tf.div(1., 1. +  tf.exp(tf.matmul(features, W) + b))
    return hypothesis

In [40]:
# loss 함수
def loss_fn(hypothesis, features, labels):
    cost = -tf.reduce_mean(labels * tf.log(logistic_regression(features) + (1 - labels) * tf.log(1 - hypothesis)))
    return cost

In [46]:
# 그래디언트 정의
def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(logistic_regression(features), features, labels)
    return tape.gradient(loss_value, [W, b])

In [53]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

EPOCHS = 1001

for step in range(EPOCHS):
    for features, labels in tfe.Iterator(dataset):
        grads = grad(logistic_regression(features), features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads, [W,b]))
        
        if step%100 == 0:
            print("Iter: {}, LOSS: {:.4f}".format(step, loss_fn(logistic_regression(features), features, labels)))

Iter: 0, LOSS: nan
Iter: 100, LOSS: nan
Iter: 200, LOSS: nan
Iter: 300, LOSS: nan
Iter: 400, LOSS: nan
Iter: 500, LOSS: nan
Iter: 600, LOSS: nan
Iter: 700, LOSS: nan
Iter: 800, LOSS: nan
Iter: 900, LOSS: nan
Iter: 1000, LOSS: nan


In [54]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.int32))
    return accuracy

In [55]:
test_acc = accuracy_fn(logistic_regression(X_test), y_test)

In [56]:
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 1.0000
